In [1]:
import multiprocessing
num_cpu = multiprocessing.cpu_count()
num_cpu

8

In [2]:
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import ZeroPadding2D, Convolution2D, MaxPooling2D, Dropout, Flatten, Activation, Dense
from tensorflow.keras.optimizers import SGD

#from keras.models import load_model
import joblib

In [3]:
tf.__version__

'2.4.0'

In [4]:
# model_faces for detected faces

model_faces = keras.Sequential()
model_faces.add(ZeroPadding2D((1,1),input_shape=(82,82,64)))
model_faces.add(Convolution2D(64, (64, 64), activation='relu'))
model_faces.add(ZeroPadding2D((1,1)))
model_faces.add(Convolution2D(64, (3, 3), activation='relu'))
model_faces.add(MaxPooling2D((2,2), strides=(2,2)))

model_faces.add(ZeroPadding2D((1,1)))
model_faces.add(Convolution2D(128, (3, 3), activation='relu'))
model_faces.add(ZeroPadding2D((1,1)))
model_faces.add(Convolution2D(128, (3, 3), activation='relu'))
model_faces.add(MaxPooling2D((2,2), strides=(2,2)))

model_faces.add(Convolution2D(1024, (5, 5), activation='relu'))
model_faces.add(Dropout(0.5))
model_faces.add(Convolution2D(1024, (1, 1), activation='relu'))
model_faces.add(Dropout(0.5))
model_faces.add(Convolution2D(2622, (1, 1)))
model_faces.add(Flatten())
model_faces.add(Dense(1, activation='sigmoid'))

model_faces.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 84, 84, 64)        0         
_________________________________________________________________
conv2d (Conv2D)              (None, 21, 21, 64)        16777280  
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 10, 10, 64)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 128)       7

In [5]:
result = open('X-24', 'rb')
X = np.load(result, allow_pickle=True)

result2 = open('Y-24', 'rb')
y = np.load(result2, allow_pickle=True)

In [6]:
print(X.shape, X.dtype)
print(y.shape, y.dtype)

(5018, 82, 82, 64) float32
(5018,) int64


In [7]:
# 80:20 for split size
size = int(len(X) * 0.8)

x_train, x_test = X[:size], X[size:]
y_train, y_test = y[:size], y[size:]

In [8]:
# To clear memory
import gc
#del model_frames
del X
del y
gc.collect()

214

In [10]:
model_faces.compile(
    optimizer=SGD(learning_rate=0.01, momentum=0.9),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=['accuracy']
)

In [11]:
history = model_faces.fit(
    x_train,
    y_train,
    batch_size=128,
    epochs=5,
    use_multiprocessing=True,
    workers=num_cpu
)

Epoch 1/5
32/32 [==============================] - 1461s 46s/step - loss: 0.6176 - accuracy: 0.6880
Epoch 2/5
32/32 [==============================] - 1479s 46s/step - loss: 0.5791 - accuracy: 0.7412
Epoch 3/5
32/32 [==============================] - 1476s 46s/step - loss: 0.5758 - accuracy: 0.7391
Epoch 4/5
32/32 [==============================] - 1474s 46s/step - loss: 0.5670 - accuracy: 0.7498
Epoch 5/5
32/32 [==============================] - 1474s 46s/step - loss: 0.5779 - accuracy: 0.7362


In [13]:
# Save tf model in pb
!mkdir -p saved_model
model_faces.save('saved_model/my_model')

INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# Save tf model
model_faces.save('model.h5')

In [ ]:
model_faces.save('model.keras')

In [ ]:
#serialize model to json
model_json = model_faces.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
model_faces.save_weights("model-with-weights.h5")